# Feature Engineering

### Importing libraries

In [15]:
import pandas as pd
import numpy as np
import os
import sys
import joblib

import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.path.abspath('..'))


In [2]:
from src.feature_engineering import (
    calculate_woe, woe_summary,
    remove_outliers_iqr,
    label_encode_columns,
    scale_columns,
    aggregate_features,
    extract_date_features
)

### Loading the Dataset

In [3]:
raw_path = "/Users/elbethelzewdie/Downloads/credit-risk-analysis/Credit-risk-analysis/data/data.csv"    # <-- change if needed

df = pd.read_csv(raw_path)
print("✅ Raw data loaded:", df.shape)
df.info()

✅ Raw data loaded: (95662, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TransactionId         95662 non-null  object 
 1   BatchId               95662 non-null  object 
 2   AccountId             95662 non-null  object 
 3   SubscriptionId        95662 non-null  object 
 4   CustomerId            95662 non-null  object 
 5   CurrencyCode          95662 non-null  object 
 6   CountryCode           95662 non-null  int64  
 7   ProviderId            95662 non-null  object 
 8   ProductId             95662 non-null  object 
 9   ProductCategory       95662 non-null  object 
 10  ChannelId             95662 non-null  object 
 11  Amount                95662 non-null  float64
 12  Value                 95662 non-null  int64  
 13  TransactionStartTime  95662 non-null  object 
 14  PricingStrategy       95662 non-null  i

In [5]:
print("📊 Data Overview")
print("=" * 50)
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

📊 Data Overview
Shape: (95662, 16)
Columns: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult']


###  Define Columns

In [ ]:
CUSTOMER_ID_COL = 'CustomerId'
AMOUNT_COL = 'Amount'  
DATE_COL = 'TransactionStartTime'
TARGET_COL = 'FraudResult' 

### Identify Numeric and Categorical Columns

In [7]:
# Identify numeric and categorical columns from your data
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

In [10]:
# Remove ID and date columns from feature lists
id_cols = [CUSTOMER_ID_COL, 'TransactionId', 'AccountId', 'SubscriptionId', 'BatchId', 'CountryCode', 'CurrencyCode']
date_cols = [DATE_COL]

### Cleaning the Feature List

In [11]:
# Clean feature lists
numeric_features = [col for col in numeric_cols 
                    if col not in id_cols + date_cols and col != TARGET_COL]
categorical_features = [col for col in categorical_cols 
                        if col not in id_cols + date_cols]

print(f"\n🔧 Column Configuration")
print(f"Customer ID column: {CUSTOMER_ID_COL}")
print(f"Amount column: {AMOUNT_COL}")
print(f"Date column: {DATE_COL}")
print(f"Target column: {TARGET_COL}")
print(f"Numeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")

# Display feature lists
if numeric_features:
    print(f"\nNumeric features: {numeric_features[:10]}{'...' if len(numeric_features) > 10 else ''}")
if categorical_features:
    print(f"Categorical features: {categorical_features[:10]}{'...' if len(categorical_features) > 10 else ''}")


🔧 Column Configuration
Customer ID column: CustomerId
Amount column: Amount
Date column: TransactionStartTime
Target column: FraudResult
Numeric features: 3
Categorical features: 4

Numeric features: ['Amount', 'Value', 'PricingStrategy']
Categorical features: ['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId']


### Check for Missing Values in Target

In [ ]:
if TARGET_COL in df.columns:
    print(f"\n🎯 Target Variable: {TARGET_COL}")
    print(f"Value counts:\n{df[TARGET_COL].value_counts()}")
    print(f"Missing values: {df[TARGET_COL].isnull().sum()}")
    
    # Check if target is binary
    if df[TARGET_COL].nunique() != 2:
        print(f"⚠️ Warning: Target has {df[TARGET_COL].nunique()} unique values. WoE requires binary target.")
else:
    print(f"⚠️ Warning: Target column '{TARGET_COL}' not found in data")


🎯 Target Variable: FraudResult
Value counts:
FraudResult
0    95469
1      193
Name: count, dtype: int64
Missing values: 0


- This shows a highly imbalanced dataset with only 193 fraud cases (0.2%) out of 95,662 total transactions, indicating we'll need special techniques like class weighting or resampling to handle the severe class imbalance.

### Remove Outliers

In [13]:
df_clean = remove_outliers_iqr(df, numeric_cols=['Amount', 'Value'])


⚠️ Removed 24441 outlier rows (25.55%)


- This indicates the outlier removal step removed 24,441 rows (over 25% of the data)

### Calculate WoE/IV

In [16]:
numerical_cols = ['Amount', 'Value']
categorical_cols = ['CurrencyCode', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId']
target_col = 'FraudResult'

iv_summary_df, woe_details = woe_summary(df_clean, target_col, numerical_cols, categorical_cols, bins=5)

# Check IV summary
print(iv_summary_df)

# Check WoE table for a feature
print(woe_details['Value_bin'])


           feature        iv
0        ProductId  3.106748
1  ProductCategory  3.020399
2       ProviderId  1.215336
3        ChannelId  1.172971
4        Value_bin  0.319927
5       Amount_bin  0.019026
6     CurrencyCode  0.000000
          Value_bin  count  event  non_event       woe  iv_contribution
0     (1.999, 50.0]  16636    0.5      16636 -0.491720         0.044615
1     (50.0, 500.0]  13629    1.0      13628  0.400870         0.037826
2   (500.0, 1000.0]  21583    0.5      21583 -0.752056         0.120475
3  (1000.0, 2000.0]   9468    1.0       9467  0.765185         0.116910
4  (2000.0, 8110.0]   9905    0.5       9905  0.026810         0.000101


- The IV results show that fraud in the dataset is driven mainly by categorical features rather than transaction size. ProductId and ProductCategory have extremely high IV (>3), while ProviderId and ChannelId are also very strong predictors, indicating fraud is concentrated around specific products, providers, and channels rather than being random. The Value feature has moderate predictive power (IV ≈ 0.32), meaning transaction size matters but only within certain ranges, whereas Amount contributes very little and CurrencyCode has no signal at all. However, IV values above 1 are unusually high in real-world data and may indicate dataset bias or leakage, so those features should be used carefully. Overall, the dataset suggests structurally patterned fraud behavior, making categorical ecosystem features more important than raw monetary values for modeling.

### Label Encode Categorical Columns

In [17]:
categorical_cols = ['CurrencyCode', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId']
df_encoded, encoders = label_encode_columns(df_clean, categorical_cols)


In [35]:
df_encoded.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,0,256,5,1,0,2,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,0,256,3,16,2,1,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,0,256,5,0,0,2,500.0,500,2018-11-15T02:44:21Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,0,256,3,16,2,1,-644.0,644,2018-11-15T03:34:21Z,2,0
5,TransactionId_23223,BatchId_25954,AccountId_1078,SubscriptionId_4238,CustomerId_1432,0,256,5,14,0,2,2000.0,2000,2018-11-15T03:35:10Z,2,0


### Scale Numeric Columns

In [18]:
numeric_cols = ['Amount', 'Value', 'PricingStrategy']
df_scaled, scalers = scale_columns(df_encoded, numeric_cols, method='standard')

In [24]:
df_scaled.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,0,256,5,1,0,2,-0.017096,-0.145205,2018-11-15T02:18:49Z,-0.286825,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,0,256,3,16,2,1,-0.651634,-0.800029,2018-11-15T02:19:08Z,-0.286825,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,0,256,5,0,0,2,-0.328144,-0.479299,2018-11-15T02:44:21Z,-0.286825,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,0,256,3,16,2,1,-1.039821,-0.383080,2018-11-15T03:34:21Z,-0.286825,0
5,TransactionId_23223,BatchId_25954,AccountId_1078,SubscriptionId_4238,CustomerId_1432,0,256,5,14,0,2,0.605000,0.522982,2018-11-15T03:35:10Z,-0.286825,0


### Aggregate Features Per Customer

In [40]:
df_agg = aggregate_features(df_scaled, customer_id_col='CustomerId', amount_col='Amount', 
                            categorical_cols=['CurrencyCode'])


In [41]:
df_agg.head()

,CustomerId,Amount_sum,Amount_mean,Amount_count,Amount_std,CurrencyCode_<lambda>
0,CustomerId_1001,1.832095,0.916047,2,2.375396,0
1,CustomerId_1002,-4.402756,-0.400251,11,0.348684,0
2,CustomerId_1003,1.192903,0.397634,3,1.713118,0
3,CustomerId_1004,0.605000,0.605000,1,NaN,0
4,CustomerId_1005,0.446388,0.148796,3,2.026670,0


### Extract Date Features

In [42]:
df_features = extract_date_features(df, date_col='TransactionStartTime')

In [43]:
df_features.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_year,transaction_dayofweek,transaction_quarter,is_weekend,is_business_hours
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,...,2,0,2,15,11,2018,3,4,0,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,...,2,0,2,15,11,2018,3,4,0,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,...,2,0,2,15,11,2018,3,4,0,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,...,2,0,3,15,11,2018,3,4,0,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,...,2,0,3,15,11,2018,3,4,0,0


### Merging Columns

In [49]:
# Ensure datetime
df_encoded['TransactionStartTime'] = pd.to_datetime(df_encoded['TransactionStartTime'], errors='coerce')

# Add time-based columns
df_encoded['transaction_hour'] = df_encoded['TransactionStartTime'].dt.hour
df_encoded['transaction_day'] = df_encoded['TransactionStartTime'].dt.day
df_encoded['transaction_month'] = df_encoded['TransactionStartTime'].dt.month
df_encoded['transaction_year'] = df_encoded['TransactionStartTime'].dt.year
df_encoded['transaction_dayofweek'] = df_encoded['TransactionStartTime'].dt.dayofweek
df_encoded['transaction_quarter'] = df_encoded['TransactionStartTime'].dt.quarter
df_encoded['is_weekend'] = df_encoded['transaction_dayofweek'].isin([5,6]).astype(int)
df_encoded['is_business_hours'] = ((df_encoded['transaction_hour'] >= 9) & (df_encoded['transaction_hour'] <= 17)).astype(int)

# Columns
numeric_cols = ['Amount', 'Value']
categorical_cols = ['ProviderId','ProductId','ProductCategory','ChannelId']
time_cols = ['transaction_hour','transaction_day','transaction_month','transaction_year',
             'transaction_dayofweek','is_weekend','is_business_hours']

# Aggregation dictionary
agg_dict = {col: ['sum','mean','count','std'] for col in numeric_cols}
agg_dict.update({col: lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0] for col in categorical_cols})
agg_dict.update({col: 'mean' for col in time_cols})

# Final aggregation
df_final = df_encoded.groupby('CustomerId').agg(agg_dict).reset_index()

# Flatten MultiIndex columns
df_final.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in df_final.columns]

df_final.head()


,CustomerId,Amount_sum,Amount_mean,Amount_count,Amount_std,Value_sum,Value_mean,Value_count,Value_std,ProviderId_<lambda>,ProductId_<lambda>,ProductCategory_<lambda>,ChannelId_<lambda>,transaction_hour_mean,transaction_day_mean,transaction_month_mean,transaction_year_mean,transaction_dayofweek_mean,is_weekend_mean,is_business_hours_mean
0,CustomerId_1001,5000.0,2500.000000,2,3818.376618,5400,2700.000000,2,3535.533906,3,1,0,1,8.000000,16.000000,11.000000,2018.000000,4.000000,0.000000,0.000000
1,CustomerId_1002,4225.0,384.090909,11,560.498966,4775,434.090909,11,518.805446,3,16,2,1,13.454545,14.727273,8.636364,2018.272727,2.909091,0.181818,0.454545
2,CustomerId_1003,5000.0,1666.666667,3,2753.785274,7000,2333.333333,3,1892.969449,5,1,0,2,14.666667,1.000000,2.000000,2019.000000,4.000000,0.000000,1.000000
3,CustomerId_1004,2000.0,2000.000000,1,NaN,2000,2000.000000,1,NaN,0,1,0,2,18.000000,22.000000,12.000000,2018.000000,5.000000,1.000000,0.000000
4,CustomerId_1005,3800.0,1266.666667,3,3257.811126,6200,2066.666667,3,2571.640203,3,16,2,1,9.666667,28.000000,12.000000,2018.000000,4.000000,0.000000,0.333333


In [52]:
# Aggregate FraudResult per customer (1 if any transaction is fraud, else 0)
fraud_agg = df_encoded.groupby('CustomerId')['FraudResult'].max().reset_index()

# Merge it into the final customer-level DataFrame
df_final_clean = df_final.merge(fraud_agg, on='CustomerId', how='left')

# Rename columns for clarity
df_final_clean.columns = [
    col.replace('<lambda>', 'mode') if '<lambda>' in col else col for col in df_final_clean.columns
]
df_final_clean.columns = [col.replace('_mean', '_avg') for col in df_final_clean.columns]

# Check final columns
print(df_final_clean.columns.tolist())

# Save to CSV
df_final_clean.to_csv('/Users/elbethelzewdie/Downloads/credit-risk-analysis/Credit-risk-analysis/data/processed/df_final_customer.csv', index=False)
print("✅ df_final_customer.csv saved with FraudResult included!")


['CustomerId', 'Amount_sum', 'Amount_avg', 'Amount_count', 'Amount_std', 'Value_sum', 'Value_avg', 'Value_count', 'Value_std', 'ProviderId_mode', 'ProductId_mode', 'ProductCategory_mode', 'ChannelId_mode', 'transaction_hour_avg', 'transaction_day_avg', 'transaction_month_avg', 'transaction_year_avg', 'transaction_dayofweek_avg', 'is_weekend_avg', 'is_business_hours_avg', 'FraudResult']
✅ df_final_customer.csv saved with FraudResult included!


In [53]:
df_final_clean.shape

(3402, 21)

- After feature engineering and preprocessing pipeline, the dataset now has 21 features.
- Originally, the raw dataset had 16 columns, so the pipeline has created many new features (aggregations, date features, outlier removal, etc.) and also applied label Encoding for categorical columns.

✅ Overall interpretation

- The feature engineering pipeline worked as intended: it expanded the dataset from 16 to 21 numeric features, removed outliers, and created meaningful features for modeling.

- High-variance features like transaction counts, sums, and coefficients of variation are likely strong predictors of credit risk.

- The absence of categorical features suggests all remaining columns are already numeric, ready for ML models like logistic regression, XGBoost, or neural networks.